First, please install the packages needed for the conversational AI:

In [ ]:
! pip install evaluate
! pip install torch
! pip install transformers
### maybe you will need to install some additional packages

# 1 Conversational question answering

### 1.1 You want to build a dense passage retriever for the question-answering task. Now you have the labeled question-answering dataset, and each sample contains the question, answer, and related passages. From the lecture, you know that we can build the negative passages by using any random passage from the dataset. Do you have any other idea to build the negative passages such that fine-tuned model gets better performance?

### 1.2 After building the training dataset for the dense passage retriever, we have B questions in a mini-batch, and each one is associated with a relevant passage. Do you have any idea to make the training effective?

### 1.3 From the lecture, we know that the system should consider the dialogue history for the conversational question-answering task. Now we have the following conversational question-answering task and want to use a GPT2 model to generate the correct answer. How will you design the input for the GPT2 model?

Jessica went to sit in her rocking chair. Today was her birthday and she was turning 80. Her granddaughter Annie was coming over in the afternoon and Jessica was very excited to see her. Her daughter Melanie and Melanie’s husband Josh were coming as well. Jessica had …

Q1: Who had a birthday?
A1: Jessica
Q2: How old would she be?
A2: 80
Q3: Did she plan to have any visitors?
A3: Yes
Q4: How many?
A4: Three
Q5: Who?

## 1.4 Dense passage retriever from Huggingface

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

####################################################################
    # TODO choose the suitable question model and tokenizer for question from huggingface
####################################################################
tokenizer_question = DPRQuestionEncoderTokenizer.from_pretrained("")
model_question = DPRQuestionEncoder.from_pretrained("")
####################################################################
    # TODO Tokenize the question and feed the tokenizer 
    # output into the question model.
    # question "what is ChatGPT ?"
####################################################################
print(embeddings_q)

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
####################################################################
    # TODO choose the suitable context model and tokenizer for context from huggingface
####################################################################
tokenizer_context = DPRContextEncoderTokenizer.from_pretrained("")
model_context = DPRContextEncoder.from_pretrained("")
####################################################################
    # TODO Tokenize the passage 1 and feed the tokenizer 
    # output into the context model.
    # passage 1 "ChatGPT is an artificial intelligence chatbot developed by OpenAI."
####################################################################
print(embeddings_p1.shape)

In [ ]:
import torch
####################################################################
    # TODO calculate the score between the user query and the passage 1
####################################################################
print('The score between the user query and the passage 1 is: ', score1)

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
####################################################################
    # TODO Tokenize the passage 2 and feed the tokenizer 
    # output into the context model.
    # passage 2 "A chatbot amis to simulate human communication."
####################################################################
print(embeddings_p2.shape)

In [ ]:
####################################################################
    # TODO calculate the score between the user query and the passage 2
####################################################################
print('The score between the user query and the passage 2 is: ', score2)

## 1.5 DualEncoder

We can also only use the query encoder to do the retrieval. What is the difference?

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
####################################################################
    # TODO choose the suitable question model and tokenizer for question from huggingface
####################################################################
tokenizer_question = DPRQuestionEncoderTokenizer.from_pretrained("")
model_question = DPRQuestionEncoder.from_pretrained("")
####################################################################
    # TODO Tokenize the question and feed the tokenizer 
    # output into the question model.
    # question "what is ChatGPT ?"
####################################################################
print(embeddings_q.shape)

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
####################################################################
    # TODO Tokenize the passage 1 and feed the tokenizer 
    # output into the question model.
    # passage 1 "ChatGPT is an artificial intelligence chatbot developed by OpenAI."
####################################################################
print(embeddings_p1.shape)

In [ ]:
import torch
####################################################################
    # TODO calculate the score between the user query and the passage 1
####################################################################
print('The score between the user query and the passage 1 is: ', score1)

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
####################################################################
    # TODO Tokenize the passage 2 and feed the tokenizer 
    # output into the question model.
    # passage 2 "A chatbot amis to simulate human communication."
####################################################################
print(embeddings_p2.shape)

In [ ]:
import torch
####################################################################
    # TODO calculate the score between the user query and the passage 2
####################################################################
print('The score between the user query and the passage 2 is: ', score2)

# 2 Chatbots

## 2.1 Open [ChatGPT](https://openai.com/blog/chatgpt) and explore some questions that ChatGPT will make the hallucination problem. What do you observe?

## 2.2 Personal Chatbot

At first, load the models and their tokenizers.

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
tokenizer = GPT2Tokenizer.from_pretrained("af1tang/personaGPT")
model = GPT2LMHeadModel.from_pretrained("af1tang/personaGPT")
if torch.cuda.is_available():
    model = model.cuda()
## utility functions ##
flatten = lambda l: [item for sublist in l for item in sublist]
print('a')

In [2]:
def to_data(x):
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()

In [3]:
def to_var(x):
    if not torch.is_tensor(x):
        x = torch.Tensor(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

In [4]:
def display_dialog_history(dialog_hx):
    for j, line in enumerate(dialog_hx):
        msg = tokenizer.decode(line)
        if j %2 == 0:
            print(">> User: "+ msg)
        else:
            print("Bot: "+msg)
            print()

In [5]:
def generate_next(bot_input_ids, do_sample=True, top_k=10, top_p=.92,
                  max_length=1000, pad_token=tokenizer.eos_token_id):
    full_msg = model.generate(bot_input_ids, do_sample=True,
                                              top_k=top_k, top_p=top_p, 
                                              max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    msg = to_data(full_msg.detach()[0])[bot_input_ids.shape[-1]:]
    return msg

Define personality of the Chatbot.

In [ ]:
personas = []
for i in range(3):
    response = input(">> Fact %d: "%(i+1))+ tokenizer.eos_token
    personas.append(response)
personas = tokenizer.encode(''.join(['<|p2|>'] + personas + ['<|sep|>'] + ['<|start|>']))

## 2.2.1 Try to talk with the defined personal chatbot

In [ ]:
# converse for 8 turns
dialog_hx = []
for step in range(4):
    # encode the user input
    user_inp = tokenizer.encode(input(">> User: ") + tokenizer.eos_token)
    # append to the chat history
    dialog_hx.append(user_inp)
        
    # generated a response while limiting the total chat history to 1000 tokens, 
    bot_input_ids = to_var([personas + flatten(dialog_hx)]).long()
    msg = generate_next(bot_input_ids)
    dialog_hx.append(msg)
    print("Bot: {}".format(tokenizer.decode(msg, skip_special_tokens=True)))

## What do you observe?

### 2.2.2 Define the action space of the personal chatbot.

In [ ]:
## available actions ##
####################################################################
    # TODO add ten actions into the action_space
    # Define the top_k, top_p, max_lenth by yourself
####################################################################
action_space = [ 'ask about age.', ]
# converse for 8 turns
dialog_hx = []
for step in range(4):
    # choose an action
    act = None
    while act not in action_space:
        display_dialog_history(dialog_hx)
        print()
        print(" actions: ")
        for k,v in enumerate(action_space): print(k,v)
        try:
            act = action_space[int(input(" input [0-10]: " ))]
        except:
            act = None
    print()
    # format into prefix code
    action_prefix = tokenizer.encode(''.join(['<|act|> '] + [act] + ['<|p1|>'] + [] + ['<|sep|>'] + ['<|start|>']))
    bot_input_ids = to_var([action_prefix + flatten(dialog_hx)]).long()
    
    # generate query conditioned on action
    msg = generate_next(bot_input_ids, top_k=top_k, top_p=top_p, max_length=max_length)
    dialog_hx.append(msg)
    
    # generate bot response
    bot_input_ids = to_var([personas+ flatten(dialog_hx)]).long()
    msg = generate_next(bot_input_ids, top_k=top_k, top_p=top_p, max_length=max_length)
    dialog_hx.append(msg)
display_dialog_history(dialog_hx)

## What do you observe?

## 3 Evaluation

### MWOZ data 
There is an example of the MWOZ dataset. 

User: I'm looking for an expensive restaurant in the centre.

System: I can recommend several restaurants in the centre of town. Are you looking for any type of food in particular.

User: Yes, Caribbean food please.

System: There is no matching restaurant. Would you like to change your search criteria?

User: What about a restaurant that serves european food?

### 3.1 Please build the dialogue state of the last user turn?

### 3.2 If we want to generate the system response by using SimpleToD , how can we build the input?

### 3.3 Now we use SimpleToD generate the following response. We also get the groundtruth from the labeled dataset. Can you evaluate them using automatic evaluation metric bleu and write the code for it?
generated sentence: There is 1 expensive modern european restaurant. Do you want me to book it for you?

groundtruth: I found 2 expensive european restaurants and 1 expensive modern european restaurant. Which kind would you prefer?

In [ ]:
from evaluate import load # use the Huggingface evaluate implementations
bleu = load("sacrebleu")
####################################################################
# TODO 
# Finish the code part
####################################################################

### 3.4 Now we want to do the human evaluation for the generated response. Could you please design the questions for the human evaluation?

## Citations
Karpukhin, V., Oguz, B., Min, S., Lewis, P., Wu, L., Edunov, S., ... & Yih, W. T. (2020, November). Dense Passage Retrieval for Open-Domain Question Answering. In Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing (EMNLP) (pp. 6769-6781).

Huggingface https://huggingface.co/ DPR model, PersonaGPT

ChatGPT Introducing ChatGPT https://openai.com/blog/chatgpt

Tang, F., Zeng, L., Wang, F., & Zhou, J. (2021). Persona authentication through generative dialogue. arXiv preprint arXiv:2110.12949.

Radford, Alec, et al. "Language models are unsupervised multitask learners." OpenAI Blog 1.8 (2019): 9.

Zhang, Yizhe, et al. "Dialogpt: Large-scale generative pre-training for conversational response generation." arXiv preprint arXiv:1911.00536 (2019).

Zhang, Saizheng, et al. "Personalizing dialogue agents: I have a dog, do you have pets too?." arXiv preprint arXiv:1801.07243 (2018).

Dinan et al., "The Second Conversational Intelligence Challenge (ConvAI2)." arXiv preprint arXiv:1902.00098 (2019).

Thomas Wolf et al. "Transfertransfo: A transfer learning approach for neural network based conversational agents." arXiv preprint328arXiv:1901.08149, 2019

MWOZ dataset https://github.com/budzianowski/multiwoz

Paweł Budzianowski, Tsung-Hsien Wen, Bo-Hsiang Tseng, Iñigo Casanueva, Stefan Ultes, Osman Ramadan, and Milica Gašić. 2018. MultiWOZ - A Large-Scale Multi-Domain Wizard-of-Oz Dataset for Task-Oriented Dialogue Modelling. In Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing, pages 5016–5026, Brussels, Belgium. Association for Computational Linguistics.

